In [ ]:
# Import libraries and packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from imutils import paths
import numpy as np
import random
import pickle
import cv2
import os
import time   # time1 = time.time(); print('Time taken: {:.1f} seconds'.format(time.time() - time1))
import warnings
from tqdm import tqdm_notebook as tqdm
import itertools

%matplotlib inline
warnings.filterwarnings("ignore")
SEED = 42   # set random seed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# change working DIR
import os
os.chdir("/content/drive/MyDrive/ANN_Projects_Extra/Customer_Churn_Prediction")

In [ ]:
!dir

In [ ]:
# # Extract dataset.rar file
# !pip install patool
# import patoolib
# patoolib.extract_archive("Customer_Churn.zip")


In [ ]:
!dir

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ANN_Projects_Extra/Customer_Churn_Prediction/Churn_Modelling.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()  #checking duplicate rows in dataframe

In [ ]:
df['Exited'].value_counts()

In [ ]:
df['Geography'].value_counts()

In [ ]:
df['Gender'].value_counts()

In [ ]:
df.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)

In [ ]:
df.head()

#Label Encoding

In [ ]:
df = pd.get_dummies(df,columns=['Geography','Gender'],drop_first=True)


In [ ]:
df.head()

In [ ]:
X = df.drop(columns=['Exited'])
y = df['Exited']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, random_state=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()

# first hidden layer has 11 nodes
model.add(Dense(units = 11, input_dim = 11, activation="relu")) #input_dim 11 beacuse 11 columns are there

 # output layer with number of possible class labels
model.add(Dense(units=1, activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
from keras.src.optimizers import Adam
#Model Compiling

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [ ]:
# train the neural network on training data set
# batch_size (32) controls the size of each group of data to pass through the network.
H = model.fit(X_train_scaled,
              y_train,

              validation_split=0.2,
              epochs=100,
              )

In [ ]:
model.predict(X_test_scaled)

In the out put we are getting the values between 0 to 1 beacuse we are using sigmoid activation function in outputlayer

In [ ]:
y_log = model.predict(X_test_scaled)

In [ ]:
y_log

In [ ]:
y_pred = np.where(y_log>0.5,1,0) #if value is > 0.5 then value is 1 otherwise it is 0

In [ ]:
def plot_confusion_metrix(y_true, y_pred,classes,
                         normalize=False,
                         title='Confusion Matrix',
                         cmap=plt.cm.Blues):
    """
    Objective
    ----------
    plot confussion matrix, classification report and accuracy score

    parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.

    classes : list
        List of labels to index the matrix

    title : title for matrix
    cmap : colormap for matrix

    returns
    ----------
   all accruacy matrix
    """


    cm = confusion_matrix(y_true,y_pred)


    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized Confusion Matrix")
    else:
        print("Confusion Matrix, Without Normalisation")


    plt.imshow(cm, interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks,classes,rotation=35)
    plt.yticks(tick_marks,classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() /2.

    for i , j in itertools.product(range(cm.shape[0]), range(cm.shape[0])):
        plt.text(j, i, format(cm[i,j], fmt),
                 horizontalalignment='center',
                 color='white' if cm[i, j] > thresh else 'black')

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # plt.tight_layout()

    plt.show()

    print("-----------------------------------------------------")
    print('Classification report')
    print(classification_report(y_true,y_pred))

    print("-----------------------------------------------------")
    acc= accuracy_score(y_true,y_pred)
    print("Accuracy of the model: ", acc)

In [ ]:
plot_confusion_metrix(y_test, y_pred,classes=["Not_Churn: 0","Churn: 1"])

In [ ]:
EPOCHS=100


In [ ]:
# plot the training and validation loss
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure(figsize = [10,8])
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("ANN: Training & Validation Loss")
plt.xlabel("Epoch #", weight="bold")
plt.ylabel("Loss", weight="bold")
plt.legend()
plt.show()

In [ ]:
# plot the training and validation accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure(figsize = [10,8])
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("ANN: Training and Validation Accuracy")
plt.xlabel("Epoch #", weight="bold")
plt.ylabel("Accuracy", weight="bold")
plt.legend()
plt.show()